# File I/O

Learn how to load and store
both individual weight vectors and entire models.

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

### Loading and Saving Tensors

For individual tensors, we can directly
invoke the `load` and `save` functions
to read and write them respectively.
Both functions require that we supply a name,
and `save` requires as input the variable to be saved.

In [2]:
x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

**Store a list of tensors and read them back into memory**

In [3]:
y = torch.zeros(4)
torch.save([x, y], 'x-file')
x2, y2 = torch.load('x-file')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

**Write and read a dictionary that maps from strings to tensors**. This is convenient when we want to read or write all the weights in a model.

In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### Loading and Saving Model Parameters

Let's start with our familiar MLP.

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

Next, we **store the parameters of the model as a file** with the name "mlp.params".

In [7]:
torch.save(net.state_dict(), 'mlp.params') # net.state_dict() contains all parameters

To recover the model, we instantiate a clone
of the original MLP model.
Instead of randomly initializing the model parameters,
we **read the parameters stored in the file directly**.

In [8]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

Since both instances have the same model parameters,
the computational result of the same input `X` should be the same.
Now verify this.

In [9]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])